 ## Prise en main de la librairie GEDCOM

### Conventions : 
Tag for Spouse FamiliyElement : 'FAMS'
$\\$
Tag for Childrens FamiliyElement : 'FAMC'

In [8]:
from gedcom.element.individual import IndividualElement
from gedcom.element.element import Element
from gedcom.parser import Parser
#from gedcom.element.element import Element
#from gedcom.element.file import FileElement
#from gedcom.element.object import ObjectElement

In [28]:
file_path = 'Queen_Eliz_II.ged'
gedcom_parser = Parser()
gedcom_parser.parse_file(file_path)
root_child_elements = gedcom_parser.get_root_child_elements()

def find_child_data(first_names,last_name):
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                if element.given_name_match(first_names):
                    L1 = []
                    L2 = []
                    (first, last) = element.get_name()
                    print("Information about " + first + " " + last + ":")
                    data = element.get_child_elements()
                    for child in data :
                        L1 += [child]
                        L2 += [child.get_tag()]
                    return L1, L2
    return "No matching person"

In [29]:
find_child_data("Elizabeth II Alexandra Mary", "Windsor")

Information about Elizabeth II Alexandra Mary Windsor:


([<gedcom.element.element.Element at 0x1207ef730>,
 ['RIN', '_UID', '_UPD', 'NAME', 'SEX', 'BIRT', 'FAMS', 'FAMC'])

In [14]:
def find_IndividualElement(first_names,last_name):
    """
    Get Individual element of ged with first names and last name

    Parameters
    ---
    first_names : str
        complete first names of the individual
    last_name : str
        last names of the individual 

    Returns 
    ---
    bool 
        Tells if the individual was found
    IndividualElement
        IndividualElement corresponding to the individual in the ged

    """
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                return (True, element)
    return [False, element]

In [36]:
def get_Spouse_FamilyElement(Individual):
    """
    Get Spouse FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMS")
    

In [32]:
def get_Childrens_FamilyElement(Individual):
    """
    Get Childrens FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMC")

In [38]:
get_Spouse_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [39]:
get_Childrens_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [44]:
A = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")
B = find_IndividualElement("George VI", "Windsor")
gedcom_parser.find_path_to_ancestor(A,B,None)

AttributeError: 'tuple' object has no attribute 'get_pointer'